In [93]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from datetime import timedelta
pd.set_option('display.max_rows',10)

## Gráfico

In [156]:
brf = yf.Ticker("BRFS3.SA").history(period='1y')
brf = brf.reset_index()

for i in ['Open','High','Close','Low']:
    brf[i] = brf[i].astype('float64')
    
fig = go.Figure(data=[go.Candlestick(x=brf['Date'],open=brf['Open'],high=brf['High'],low=brf['Low'],close=brf['Close'])])
#fig.show()

## Cálculo do vol acumulado médio - periodo 60d, interval 5m

In [296]:
pd.set_option('display.max_rows',2000)

def volume_m5(tick):
    m5 = yf.Ticker(tick+".SA").history(period="60d",interval="2m", prepost = True).reset_index()
    #trata horário de verão
    m5.loc[m5.Datetime.dt.date < pd.to_datetime("2021-02-22").date() , 'Datetime']  = (m5.Datetime - timedelta(minutes=60))
    m5.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
    m5['Volume']=m5.Volume/100#1000000
    m5['vol_acumulado'] = m5.groupby(m5.Datetime.dt.date)['Volume'].cumsum()
    m5['hour']=m5.Datetime.dt.hour
    m5['minute']=m5.Datetime.dt.minute
    return m5

In [ ]:
ibr=pd.read_csv('./ibr.txt', header = None)
for index, row in ibr.iterrows():
    df = volume_m5(row[0])
#df

In [229]:
brkm = yf.Ticker("BRKM5.SA").history(period="60d",interval="2m", prepost = True).reset_index().tail(3)
lwsa = yf.Ticker("LWSA3.SA").history(period="60d",interval="2m", prepost = True).reset_index().tail(3)
c = pd.concat([brkm,lwsa])

## Merge do vol

In [212]:
m5_media_qtd = pd.merge(
    m5.groupby([(m5.hour),(m5.minute)])['vol_acumulado'].mean().to_frame().round(2).reset_index().rename(columns={'vol_acumulado':'vol_acum_medio'})
    ,m5.groupby([(m5.hour),(m5.minute)])['vol_acumulado'].count().to_frame().round(2).reset_index().rename(columns={'vol_acumulado':'qtd_vol_acum'})
    ,how='inner', on=['hour','minute']
         )
#m5_media_qtd = m5_media_qtd[m5_media_qtd.qtd_vol_acum > m5_media_qtd.qtd_vol_acum.max()*0.2]
m5_merge = pd.merge(m5,m5_media_qtd,how='left',on=['hour','minute'])
m5_merge['pct_vol_acumulado'] = m5_merge.vol_acumulado / m5_merge.vol_acum_medio*100
m5_merge=m5_merge.round(2)
m5_merge.to_csv('./m5.csv')
m5_merge[(m5_merge.hour == 10) & (m5_merge.minute == 42) & (m5_merge.pct_vol_acumulado > 110)]

,Datetime,Open,High,Low,Close,Volume,vol_acumulado,hour,minute,vol_acum_medio,qtd_vol_acum,pct_vol_acumulado
224,2021-01-07 10:42:00-02:00,24.55,24.67,24.54,24.67,243.0,9659.0,10,42,4607.31,32,209.65
636,2021-01-11 10:42:00-02:00,24.40,24.43,24.40,24.40,50.0,35098.0,10,42,4607.31,32,761.79
1047,2021-01-13 10:42:00-02:00,25.79,25.94,25.76,25.94,569.0,7002.0,10,42,4607.31,32,151.98
3510,2021-02-01 10:42:00-02:00,24.70,24.80,24.70,24.80,119.0,5458.0,10,42,4607.31,32,118.46
3715,2021-02-02 10:42:00-02:00,26.40,26.49,26.40,26.40,246.0,7724.0,10,42,4607.31,32,167.65
3921,2021-02-03 10:42:00-02:00,27.79,27.83,27.78,27.83,116.0,6360.0,10,42,4607.31,32,138.04
6098,2021-02-22 10:42:00-03:00,28.59,28.85,28.59,28.85,376.0,6720.0,10,42,4607.31,32,145.86
6561,2021-02-24 10:42:00-03:00,31.85,31.89,31.61,31.64,347.0,7634.0,10,42,4607.31,32,165.69


## Diário

In [204]:
acao="BRFS3.SA"
df = yf.Ticker(acao).history(period="30d")
df['ultima_cotacao'] = yf.Ticker(acao).history(period="1m").tail(1).Close.iloc[0]
df['cotacao_dia_anterior'] = df['Close'].shift(1)
df['pct_do_dia'] = (df.Close - df.cotacao_dia_anterior) / df.cotacao_dia_anterior*100
df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
df.Volume = df.Volume/1000000
df['media_vol'] = df.Volume.mean()
df['var_vol'] = df.Volume / df.media_vol*100
df['pct_longo'] = (df.ultima_cotacao-df.Low)/df.Close*100
df=df.reset_index()
#df.sort_index(ascending=False).round(2)
df = df.sort_values(by='Date',ascending=True).round(2)
df[df.var_vol > 100]

,Date,Open,High,Low,Close,Volume,ultima_cotacao,cotacao_dia_anterior,pct_do_dia,media_vol,var_vol,pct_longo
0,2021-01-11,21.74,21.86,21.42,21.53,9.91,23.86,NaN,NaN,8.46,117.14,11.33
1,2021-01-12,21.65,22.29,21.31,21.36,13.41,23.86,21.53,-0.79,8.46,158.51,11.94
2,2021-01-13,21.40,21.56,20.93,21.24,12.08,23.86,21.36,-0.56,8.46,142.73,13.79
4,2021-01-15,21.00,21.27,20.62,20.76,9.55,23.86,21.10,-1.61,8.46,112.88,15.61
8,2021-01-21,20.42,20.56,19.86,20.05,8.97,23.86,20.41,-1.76,8.46,105.96,19.95
9,2021-01-22,19.85,20.72,19.63,20.69,9.57,23.86,20.05,3.19,8.46,113.03,20.44
10,2021-01-26,20.63,20.79,20.37,20.41,9.41,23.86,20.69,-1.35,8.46,111.20,17.10
11,2021-01-27,20.38,21.33,20.24,21.02,10.12,23.86,20.41,2.99,8.46,119.58,17.22
12,2021-01-28,20.92,21.78,20.77,21.65,9.80,23.86,21.02,3.00,8.46,115.84,14.27
13,2021-01-29,21.52,22.24,21.22,21.22,10.74,23.86,21.65,-1.99,8.46,126.90,12.44
